In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [3]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [7]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='6a1f331b-8bc2-47c3-8c37-2b3105189fca'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is green". The system likely wants to update the favourite colour in the state. There\'s a function update_favourite_colour. We should call it with favourite_colour: "green". Then respond acknowledging.', 'tool_calls': [{'id': 'fc_b229384a-fd71-4a6b-914d-1e83c73392ea', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 137, 'total_tokens': 215, 'completion_time': 0.165706327, 'completion_tokens_details': {'reasoning_tokens': 47}, 'prompt_time': 0.009666696, 'prompt_tokens_details': None, 'queue_time': 0.042947956, 'total_time': 0.175373023}, 'model_name'

In [8]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='fa8b3f58-1861-4514-8e15-f3b4cf920fd1'),
              AIMessage(content="Hello! I'm doing great, thanks for asking. How can I assist you today?", additional_kwargs={'reasoning_content': 'We need to respond. The user says hello. We can respond politely. No need to call function.'}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 138, 'total_tokens': 186, 'completion_time': 0.099628559, 'completion_tokens_details': {'reasoning_tokens': 22}, 'prompt_time': 0.006574805, 'prompt_tokens_details': None, 'queue_time': 0.00442592, 'total_time': 0.106203364}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_178e424650', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c49a4-baa4-7183-a638-bd23d96791e1-0', tool_calls=[], invalid_tool_calls=[], usage_metadat

## Read state

In [9]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [10]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='d018f162-166e-4659-bf91-71df564cb836'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is green". We need to store this in state using update_favourite_colour. Then maybe respond acknowledging. Use function.', 'tool_calls': [{'id': 'fc_53868f38-a577-4a3a-a529-58b2fb8ca676', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 161, 'total_tokens': 224, 'completion_time': 0.130325992, 'completion_tokens_details': {'reasoning_tokens': 32}, 'prompt_time': 0.006057837, 'prompt_tokens_details': None, 'queue_time': 0.004930339, 'total_time': 0.136383829}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8634b911ff', 'service_tier': 'o

In [11]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='d018f162-166e-4659-bf91-71df564cb836'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "My favourite colour is green". We need to store this in state using update_favourite_colour. Then maybe respond acknowledging. Use function.', 'tool_calls': [{'id': 'fc_53868f38-a577-4a3a-a529-58b2fb8ca676', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 161, 'total_tokens': 224, 'completion_time': 0.130325992, 'completion_tokens_details': {'reasoning_tokens': 32}, 'prompt_time': 0.006057837, 'prompt_tokens_details': None, 'queue_time': 0.004930339, 'total_time': 0.136383829}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8634b911ff', 'service_tier': 'o